<h1>Содержание <span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Выбор-локации-для-скважины" data-toc-modified-id="Выбор-локации-для-скважины-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Выбор локации для скважины</a></span></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи**:
1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

# Загрузка и подготовка данных

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from sklearn.preprocessing import StandardScaler

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [79]:
#Загрузка данных и первичный анализ
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

display(df_0.describe())

display(df_1.describe())

display(df_2.describe())

# Проверка на корреляцию
display(df_0[['f0','f1','f2']].corr())

display(df_1[['f0','f1','f2']].corr())

display(df_2[['f0','f1','f2']].corr())

#Сборка данных в 1
data = pd.concat([df_0, df_1, df_2])
data = data.reset_index()

print()
print('Кол-во дубликатов:',data.duplicated().sum())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


,f0,f1,f2
f0,1.000000,-0.440723,-0.003153
f1,-0.440723,1.000000,0.001724
f2,-0.003153,0.001724,1.000000


,f0,f1,f2
f0,1.000000,0.182287,-0.001777
f1,0.182287,1.000000,-0.002595
f2,-0.001777,-0.002595,1.000000


,f0,f1,f2
f0,1.000000,0.000528,-0.000448
f1,0.000528,1.000000,0.000779
f2,-0.000448,0.000779,1.000000



Кол-во дубликатов: 0


**Вывод:** 
1. Загрузили данные и все необходимые библиотеки
2. Проверили данные на наличие дубликатов и корреляцию. Дубликаты и пропукси - отсутствуют. В df_0, признак f1 имеет сильную отрицательную корреляцию, но так как мы не хнаем, что это за данные, удалять признаки мы не будем.

# Обучение и проверка модели

In [80]:
array_name = ['0',
              '1',
              '2']

array_frame = [df_0,
              df_1,
              df_2]

data_dict = {'models':{},'features_target_valid':{}}

for frame, name in zip(array_frame,array_name):
    target = frame['product']
    features = frame.drop(['product','id'] , axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5 
    print('RMSE модели линейной регрессии на валидационной выборке в Pегионе',name,':', rmse)
    print('Средний запас предсказанного сырья:',predictions_valid.mean())
    
    # Создание датафрейма по Valid
    data_dict['models'][name] = model
    data_dict['features_target_valid'][name] = (features_valid,target_valid)
    
model_0 = data_dict['models']['0']
model_1 = data_dict['models']['1']
model_2 = data_dict['models']['2']
    
predicted_0 = model_0.predict(data_dict['features_target_valid']['0'][0])
predicted_1 = model_1.predict(data_dict['features_target_valid']['1'][0])
predicted_2 = model_2.predict(data_dict['features_target_valid']['2'][0])


RMSE модели линейной регрессии на валидационной выборке в Pегионе 0 : 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438038
RMSE модели линейной регрессии на валидационной выборке в Pегионе 1 : 0.893099286775616
Средний запас предсказанного сырья: 68.728546895446
RMSE модели линейной регрессии на валидационной выборке в Pегионе 2 : 40.02970873393434
Средний запас предсказанного сырья: 94.96504596800489


**Вывод:** 
Оценим регионы от самого лучшего к хучшему:
1. В регионе d_0 RMSE - самая маленькая среди трех (0.37), по среднему объему запасов сырья регион на втором месте(92).
2. В регионе d_2 RMSE = 0.40, средний запас сырья = 94.
3. В регионе d_1 RMSE высокая = 0.80, средний запас сырья = 68

*Итог:* нам больше всего подходят 0 и 2 регионы, так как у них самые лучшие показатели.

# Подготовка к расчёту прибыли

In [81]:
BUDGET = 10*(10**9)
MAX_WELLS = 500
DEV_WELLS = 200
REV_BARREL = 450000 
LOSS_MAX = 0.025
BARREL_PROD_VOL = (BUDGET/(REV_BARREL*1000))/(DEV_WELLS)

print("Достаточный объём сырья для безубыточной разработки скважины:",round(BARREL_PROD_VOL,3))
print("Средний запас в скважине в Регионе 0:",round(df_0['product'].mean(),3))
print("Средний запас в скважине в Регионе 1:",round(df_1['product'].mean(),3))
print("Средний запас в скважине в Регионе 2:",round(df_2['product'].mean(),3))

Достаточный объём сырья для безубыточной разработки скважины: 0.111
Средний запас в скважине в Регионе 0: 92.5
Средний запас в скважине в Регионе 1: 68.825
Средний запас в скважине в Регионе 2: 95.0


**Вывод:**
1. Выписали все необходимые переменные, посчитали достаточный объем сырья для безубыточной разработки скважины = 111.
2. Средний запас в скважине в регионах (в тысячах баррелей): 92 (d_0), 69 (d_1), 95 (d_2)

**Итог** Средний запас во всех регионах не превышает минимальный объем сырья для разработки скважин.

# Расчёт прибыли и рисков 

In [82]:
def profit(target, predicted):
    probs_sorted = pd.Series(predicted).sort_values(ascending=False)[:DEV_WELLS]
    true_target_sort = (target
                         .reset_index(drop = True)[probs_sorted.index])
    sum_true = true_target_sort.sum() 
    return round((sum_true * REV_BARREL) - BUDGET)
    
print("Прибыль в Pегионе 0:",profit(data_dict['features_target_valid']['0'][1],
                                                               predicted_0))
print("Прибыль в Pегионе 1:",profit(data_dict['features_target_valid']['1'][1],
                                                               predicted_1))
print("Прибыль в Pегионе 2:",profit(data_dict['features_target_valid']['2'][1],
                                                               predicted_2))


Прибыль в Pегионе 0: 3320826043
Прибыль в Pегионе 1: 2415086697
Прибыль в Pегионе 2: 2710349964


**Вывод:**
1. Самый прибыльный регион - регион 2.
2. На втором месте - регион 0.
3. На третьем месте - регион 1.


In [84]:
state = np.random.RandomState(12345)
def bootstrap(target,pred):
    samples = []
    for i in range(1000):
        sample = pd.Series(pred).sample(n = 500, replace=True, random_state=state)
        samples.append(profit(target,sample))
    samples = pd.Series(samples)
    print("Средняя прибыль =",samples.mean())
    print('Риск =',(samples < 0).mean()*100,"%")
    lower = samples.quantile(0.025)
    upper = samples.quantile(0.975)
    return round(lower,2), round(upper,2)
print()
print('Регион 0:')
print('95% Confidence interval =',bootstrap(data_dict['features_target_valid']['0'][1],pd.Series(predicted_0)))
print()
print('Регион 1:')
print('95% Confidence interval =',bootstrap(data_dict['features_target_valid']['1'][1],pd.Series(predicted_1)))
print()
print('Регион 2:')
print('95% Confidence interval =',bootstrap(data_dict['features_target_valid']['2'][1],pd.Series(predicted_2)))


Регион 0:
Средняя прибыль = 396164984.807
Риск = 6.9 %
95% Confidence interval = (-111215545.6, 909766941.5)

Регион 1:
Средняя прибыль = 461155817.281
Риск = 0.7000000000000001 %
95% Confidence interval = (78050810.35, 862952060.0)

Регион 2:
Средняя прибыль = 392950475.171
Риск = 6.5 %
95% Confidence interval = (-112227625.75, 934562914.7)


**Вывод:** После проведения бутстрэппинга, выяснилось:
1. Из 3х регионов самая высокая средняя прибыль в Регионе 1.
2. Самый низкий экономический  риск найден в Регионе 1.
3. В регионе 1 самый маленький confidence interval, так что вероятность наличия истинного среднего значения мала.

# Общий вывод

Проведя исследование по имеющимся скважинам можно сделать следующие выводы:

1. По условию задачи, по требованиям максимальной вероятности убытков в 2.5% нам подходит Регион 1.
   а) Самая маленькая вероятность рисков найдена в регионе 1 (0.7%)
   б) На втором месте по рискам - регион 2 (6.5%)
   в) Самые высокие экономические риски в регионе 0 (6.9%)
2. В Регионе 1 также самая высокая средняя прибыль: 461155817 рублей
3. Однако, по общей прибыли в регионах:
   а) Регион 0 занимает 1 место 3320826043 руб
   б) Регион 2 занимает 2 место 2710349964 руб
   в) Регион 1 занимает 3 место 2415086697 руб
   Что говорит о возможных выбросах в данных.
   
Итог: Инвестировать стоит в Регион 1

Рекоммендации: Возможно следует провести дополнительный анализ по региону 1 и проверить его на выбросы в данных.
